### Homework 8 for Computational Economics, ECON-GA 3002

**{Name, NYUID, Email}**: Arnav Sood, N11193569, asood@nyu.edu

#### Exercise 1:

> If $S$ is a discrete set, show that if the set of distributions on $S$, $\mathbb{P}(S)$, is compact in $S$, then $S$ is finite.

**Proof**: Argue contrapositively: that is, show $|S| = |\mathbb{N}| = \aleph_{0} \implies \mathbb{P}(S)$ not compact. Compactness requires that every sequence in a subset converges to a point in that subset. Let $K$ be the set of delta distributions $(1, 0, 0, ...), (0, 1, 0, ...)$, and so on. These form a subset of $S$, but clearly do not converge, as the $l_1$  norm of their difference is always 2. Therefore, $\mathbb{P}(S)$ compact shows $S$ finite for discrete $S$.


#### Exercise 2:

> Show that the deterministic Markov chain given by $X_{t+1} = X_t + 1$ has no stationary distribution.

**Proof:** We can uniquely identify each state $\gamma \in \mathbb{Z}$ with a probability distribution $p_{\gamma} \in \mathbb{P}(S)$. This is because $p_{\gamma}$ is the dirac distibrution which assigns mass 1 to $\gamma + 1$ and weight 0 everywhere else, and $\gamma + 1$ is uniquely identified with $\gamma$. So, a stationary distribution which maps to itself would correspond to a state $\gamma$ which maps to itself, which would require $\gamma + 1 = \gamma$, which is a contradiction. Therefore, no stationary distribution exists.

#### Exercise 3:

> Consider a Markov process given by the following dynamics. Select an integer $Q$, and an integer $0 \leq q \leq Q$. At time $t$, a store's inventory $X_t$ is some integer. The store then orders inventory $Q - X_t$ subject to a threshold rule: if $X_t \leq q$ the store orders $Q - X_t$, and if not the store orders nothing. The store is then subject to a stochastic demand, IID with $\text{Pr}[D = d]$ given by $\frac{1}{2}^{d+1}$ for $d \in \{0, 1, ...\}$. The store meets the demand as best it can without exceeding its current stock. The remainder, zero or nonzero, forms the next period's inventory $X_{t+1}$. 

> Show that the above is globally stable $\forall Q, 0 \leq q \leq Q$.

**Proof**: Take any finite $Q$. Setting $0 \leq q \leq Q$ yields a finite state space of cardinality $Q+1$. The associated Markov chain is irreducible, as given any initial state $\gamma$, there is positive probability on the event of selling all one's inventory, then selling some $Q-\epsilon$ for arbitrary $\epsilon \in \{0, 1, ..., Q\}$.

It is also aperiodic, as any state is accessible from itself in any number of steps greater than 3. The associated probability is, at least, the probability associated with selling everything, then not selling anything until time $k-1$, until you sell $Q-\gamma$ for initial state $\gamma$, giving you inventory $\gamma$ at time $k$.

Since the Markov chain associated with $Q, q$ is aperiodic and irreducible on a finite state space, we know it is globally stable.


#### Exercise 4

> Let $\psi$ be the stationary distribution corresponding to $(Q, q) = (5, 2)$. Using QuantEcon's $\texttt{MarkovChain}$ class, compute $\psi$.

> Import dependencies and define primitives. We take care to use `const` wherever possible, and write type-stable functions.

In [1]:
using QuantEcon
const q = 2
const Q = 5
global P = zeros(Q+1,Q+1)

6x6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

> Write a function to compute the transition probability $p(x,y)$ for all $x$, and nonzero $y$. The reason that we restrict to nonzero $y$ is that the 0 inventory case corresponds to an infinite number of states (i.e., all $d > Q$), and so we compute that probability by subtracting those that we can explicitly compute from 1.

In [2]:
function transitionProbability(x,y)
    if x <= q
        return (0.5)^(Q-y+1)
    elseif (x > q) & (y <= x)
        return (0.5)^(x-y+1)
    else 
        return 0
    end
end

transitionProbability (generic function with 1 method)

> We write a function to generate the stochastic kernel, given our transition function.

In [3]:
function getStochasticMatrix(P)
    for row in 1:Q+1
        for column in 2:Q+1
            P[row,column] = transitionProbability(row-1,column-1)
        end
    end
    
    for row in 1:Q+1
        P[row,1] = 1 - sum(P[row,:])
    end
    
    return P
end

getStochasticMatrix (generic function with 1 method)

> Generate our stochastic matrix, wrap it in a `MarkovChain` object, and then compute the stationary distribution.

In [4]:
stochasticKernel = getStochasticMatrix(P)
inventoryChain = MarkovChain(stochasticKernel)
println("The stochastic kernel is:", stochasticKernel)
println("The stationary distribution is:", mc_compute_stationary(inventoryChain))

The stochastic kernel is:[0.03125 0.03125 0.0625 0.125 0.25 0.5
 0.03125 0.03125 0.0625 0.125 0.25 0.5
 0.03125 0.03125 0.0625 0.125 0.25 0.5
 0.125 0.125 0.25 0.5 0.0 0.0
 0.0625 0.0625 0.125 0.25 0.5 0.0
 0.03125 0.03125 0.0625 0.125 0.25 0.5]
The stationary distribution is:[0.0625,0.0625,0.125,0.25,0.25,0.25]


#### Exercise 5

> Compute the stationary distribution from above, but use an explicit iteration on the $l_1$ norm instead of a package from QuantEcon.

> Define an initial distribution, here the uniform one. Because we intend to use this variable name again, this one will be distinguished by exercise. 

> The transpose is to give us a row vector, and the rational type from Julia lets us configure things properly for an arbitrary $Q$.

In [5]:
const initialDist_ex5 = (zeros(Q+1)+1//(Q+1))'

1x6 Array{Float64,2}:
 0.166667  0.166667  0.166667  0.166667  0.166667  0.166667

> Write a function to compute the $l_1$ distance of two distributions.

In [6]:
function computeNorm(dist1,dist2)
    newDist = abs(dist1-dist2)
    l1 = sum(newDist)
    return l1
end

computeNorm (generic function with 1 method)

> Write a loop to iterate on the $l_1$ norm up to $10^{-4}$. We also define some constants to be used inside the loop.

In [7]:
currentDist = initialDist_ex5
diffNorm = computeNorm(currentDist*P, currentDist)

while diffNorm > 0.0001
    nextDist = currentDist*P
    diffNorm = computeNorm(nextDist,currentDist)
    currentDist = nextDist
end

> Print the iterated distribution.

In [8]:
println("The stationary distribution is: ", currentDist)

The stationary distribution is: [0.06250251395006973 0.06250251395006973 0.12500502790013945 0.2500100558002789 0.24998949468135828 0.24999039371808365]


> We can confirm that this looks quite close to the analytically true distribution. 